In [31]:
import json
import os
import xml.etree

from biryani.baseconv import check
import openfisca_core
import openfisca_france
from openfisca_core import decompositionsxml

In [32]:
tax_benefit_system = openfisca_france.init_country()

In [53]:
def xml_to_json(tree):
    xml_json = check(decompositionsxml.xml_decomposition_to_json)(tree.getroot())
    xml_json = check(decompositionsxml.make_validate_node_xml_json(tax_benefit_system))(xml_json)
    decomposition_json = decompositionsxml.transform_node_xml_json_to_json(xml_json)
    return decomposition_json

In [54]:
xml_file_path = os.path.join(tax_benefit_system.DECOMP_DIR, tax_benefit_system.DEFAULT_DECOMP_FILE)
tree = xml.etree.ElementTree.parse(xml_file_path)
decomposition_json = xml_to_json(tree)
print json.dumps(decomposition_json, indent = 2)

{
  "@context": "http://openfisca.fr/contexts/decomposition.jsonld", 
  "@type": "Node", 
  "color": [
    0, 
    0, 
    255
  ], 
  "short_name": "Rev. disp.", 
  "code": "revdisp", 
  "type": 2, 
  "name": "Revenu disponible", 
  "children": [
    {
      "@type": "Node", 
      "color": [
        0, 
        155, 
        204
      ], 
      "short_name": "Rev. nets", 
      "code": "rev_net", 
      "name": "Revenus nets", 
      "children": [
        {
          "@type": "Node", 
          "color": [
            0, 
            155, 
            204
          ], 
          "short_name": "Rev. travail", 
          "code": "rev_trav", 
          "name": "Revenu du travail", 
          "children": [
            {
              "@type": "Node", 
              "color": [
                0, 
                138, 
                201
              ], 
              "short_name": "Sal. nets", 
              "code": "salnet", 
              "name": "Salaires nets", 
              "childr

In [108]:
def transform_node_xml_json_to_julia_list_tree(node_xml_json):
    variable_name = node_xml_json['code']
    children_variables_name = map(transform_node_xml_json_to_julia_list_tree, node_xml_json['NODE']) \
        if node_xml_json.get('NODE') \
        else None
    return [variable_name, children_variables_name]

In [143]:
def transform_julia_list_tree_to_julia_source_code(node, depth = 0):
    return u'{depth}@decomposition_node {name} {children}'.format(
        children = u'[]\n'
            if node[1] is None
            else u'[\n{children}{depth}]\n'.format(
            children = ''.join(
                transform_julia_list_tree_to_julia_source_code(child_node, depth + 1)
                for child_node in node[1]
                ),
            depth = ' ' * 2 * depth,
            ),
        depth = ' ' * 2 * depth,
        name = node[0],
        )

In [148]:
def xml_to_julia(tax_benefit_system, tree):
    xml_json = check(decompositionsxml.xml_decomposition_to_json)(tree.getroot())
    xml_json = check(decompositionsxml.make_validate_node_xml_json(tax_benefit_system))(xml_json)
    julia_list_tree = transform_node_xml_json_to_julia_list_tree(xml_json)
    julia_source_code = transform_julia_list_tree_to_julia_source_code(julia_list_tree)
    return julia_source_code

In [149]:
xml_file_path = os.path.join(tax_benefit_system.DECOMP_DIR, tax_benefit_system.DEFAULT_DECOMP_FILE)
tree = xml.etree.ElementTree.parse(xml_file_path)
decomposition_julia = xml_to_julia(tax_benefit_system, tree)
print decomposition_julia

@decomposition_node revdisp [
  @decomposition_node rev_net [
    @decomposition_node rev_trav [
      @decomposition_node salnet [
        @decomposition_node sal [
          @decomposition_node salbrut [
            @decomposition_node salsuperbrut []
            @decomposition_node cotisations_patronales []
            @decomposition_node allegement_fillon []
            @decomposition_node credit_impot_competitivite_emploi []
            @decomposition_node tehr []
          ]
          @decomposition_node cotisations_salariales []
          @decomposition_node csgsald []
          @decomposition_node mhsup []
        ]
        @decomposition_node csgsali []
        @decomposition_node crdssal []
        @decomposition_node forfait_social []
        @decomposition_node hsup []
      ]
      @decomposition_node rag []
      @decomposition_node ric []
      @decomposition_node rnc []
    ]
    @decomposition_node pen [
      @decomposition_node chonet [
        @decomposition_node ch

In [147]:
with open('./decompositions.jl', 'w') as decompositions_file:
    decompositions_file.write(decomposition_julia)